# **Mask generation**

This notebook shows how the masks were generated.

The notebook is based on Abhiroop Talasila. Generating Image Segmentation Masks — The Easy Way. Oct. 1, 2020. URL: https://towardsdatascience.com/generating-image-segmentation-masks-the-easy-way-dd4d3656dbd1 (visited on 04/19/2022).


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import cv2
import json
import numpy as np

In [ ]:
!cp -R /content/gdrive/MyDrive/video_frames /

In [ ]:
!cp -R /content/gdrive/MyDrive/masks /

In [ ]:
from PIL import Image

In [ ]:
file_bbs = {}

In [ ]:
def add_to_dict(data, itr, key, count):
    try:
	    x_points = data[itr]["regions"][count]["shape_attributes"]["all_points_x"]
	    y_points = data[itr]["regions"][count]["shape_attributes"]["all_points_y"]
    except:
      print("No BB. Skipping", key)
      return
    
    all_points = []
    for i, x in enumerate(x_points):
        all_points.append([x, y_points[i]])
    
    file_bbs[key] = all_points

In [ ]:
def generate_mask_images(folder, class_label, json_file):
  # Read JSON file
  with open(os.path.join('/masks', class_label, folder,json_file)) as f:
    data = json.load(f)
  for itr in data:
    file_name_json = data[itr]["filename"]
    sub_count = 0               # Contains count of masks for a single ground truth image
    
    if len(data[itr]["regions"]) > 1:
        for _ in range(len(data[itr]["regions"])):
            key = file_name_json[:-4] + "*" + str(sub_count+1)
            add_to_dict(data, itr, key, sub_count)
            sub_count += 1
    else:
        add_to_dict(data, itr, file_name_json[:-4], 0)
  
  print("\nDict size: ", len(file_bbs))
  
  count = 0
  source_image_folder = os.path.join('/video_frames',class_label, folder)
  mask_folder = os.path.join('/masks',class_label, folder, 'masks')
  if not os.path.exists(mask_folder):
    os.mkdir(mask_folder)
  for itr in file_bbs:
    num_masks = itr.split("*")
    

    im = Image.open(os.path.join(source_image_folder,f'{itr}.jpg'))
    MASK_WIDTH, MASK_HEIGHT = im.size

    mask = np.zeros((MASK_HEIGHT, MASK_WIDTH))
    try:
        arr = np.array(file_bbs[itr])
    except:
        print("Not found:", itr)
        continue
    count += 1
    cv2.fillPoly(mask, [arr], color=(255))
    
    if len(num_masks) > 1:
      cv2.imwrite(os.path.join(mask_folder, itr.replace("*", "_") + ".png") , mask)
      #print(os.path.join(mask_folder, itr.replace("*","_")+".png"))
    else:
      cv2.imwrite(os.path.join(mask_folder, itr + ".png") , mask)
      #print(os.path.join(mask_folder, itr.replace("*","_")+".png"))
        
  print("Images saved:", count)
  file_bbs.clear()



In [ ]:
for cl in os.listdir('/masks'):
  if 'PizzaTossing' not in cl:
    for dir in os.listdir(os.path.join('/masks', cl)):
      for json_file in os.listdir(os.path.join('/masks',cl,dir)):
        generate_mask_images(dir, cl, 'frame_0.json')


print('Generating PizzaTossing Masks')
generate_mask_images('v_PizzaTossing_g02_c02', 'PizzaTossing', 'frame_0.json')
generate_mask_images('v_PizzaTossing_g02_c02', 'PizzaTossing', 'frame_10.json')
print('Done')

print('Generating PlayingGuitar Masks')
generate_mask_images('v_PlayingGuitar_g06_c05', 'PlayingGuitar', 'frame_0.json')
generate_mask_images('v_PlayingGuitar_g11_c02', 'PlayingGuitar', 'frame_0.json')
generate_mask_images('v_PlayingGuitar_g22_c03', 'PlayingGuitar', 'frame_0.json')
print('Done')

print('Generating PushUps Masks')
generate_mask_images('v_PushUps_g11_c03', 'PushUps', 'frame_0.json')
generate_mask_images('v_PushUps_g14_c04', 'PushUps', 'frame_0.json')
generate_mask_images('v_PushUps_g18_c02', 'PushUps', 'frame_0.json')
print('Done')

print('Generating SalsaSpin Masks')
generate_mask_images('v_SalsaSpin_g04_c02', 'SalsaSpin', 'frame_0.json')
generate_mask_images('v_SalsaSpin_g11_c02', 'SalsaSpin', 'frame_0.json')
generate_mask_images('v_SalsaSpin_g18_c04', 'SalsaSpin', 'frame_0.json')
print('Done')

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
No BB. Skipping frame_66
No BB. Skipping frame_67
No BB. Skipping frame_68
No BB. Skipping frame_69
No BB. Skipping frame_70
No BB. Skipping frame_71
No BB. Skipping frame_72
No BB. Skipping frame_73
No BB. Skipping frame_74
No BB. Skipping frame_75
No BB. Skipping frame_76
No BB. Skipping frame_77
No BB. Skipping frame_78
No BB. Skipping frame_79
No BB. Skipping frame_80
No BB. Skipping frame_81
No BB. Skipping frame_82
No BB. Skipping frame_83
No BB. Skipping frame_84
No BB. Skipping frame_85
No BB. Skipping frame_86
No BB. Skipping frame_87
No BB. Skipping frame_88
No BB. Skipping frame_89
No BB. Skipping frame_90
No BB. Skipping frame_91
No BB. Skipping frame_92

Dict size:  51
Images saved: 51
No BB. Skipping frame_51
No BB. Skipping frame_52
No BB. Skipping frame_53
No BB. Skipping frame_54
No BB. Skipping frame_55
No BB. Skipping frame_56
No BB. Skipping frame_57
No BB. Skipping frame_58
No BB. Skipping frame_59


In [ ]:
!ls /masks/SalsaSpin/

In [ ]:
!cp -R /masks/ /content/gdrive/MyDrive/masked_images_backup/